In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import os

# Define the path to your folder
train_dir = '/content/drive/My Drive/Third Year/Spring Quarter/POLI 179/UCLA-protest/img/train'
test_dir = '/content/drive/My Drive/Third Year/Spring Quarter/POLI 179/UCLA-protest/img/test'

# Check if the paths exist
print("Train directory exists:", os.path.exists(train_dir))
print("Test directory exists:", os.path.exists(test_dir))


Train directory exists: True
Test directory exists: True


In [ ]:
!cp -r '/content/drive/My Drive/Third Year/Spring Quarter/POLI 179/UCLA-protest/img/train' /content/train
!cp -r '/content/drive/My Drive/Third Year/Spring Quarter/POLI 179/UCLA-protest/img/test' /content/test

# Update paths to the local filesystem
train_dir = '/content/train'
test_dir = '/content/test'

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_and_preprocess_image(path, target_size=(150, 150)):
    img = load_img(path, target_size=target_size)
    img = img_to_array(img)
    img = img / 255.0  # Normalize to [0, 1]
    return img

def load_images_from_directory(directory, label_value):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            img_path = os.path.join(directory, filename)
            img = load_and_preprocess_image(img_path)
            images.append(img)
            labels.append(label_value)
    return np.array(images), np.array(labels)

# Load train and test images
train_images, train_labels = load_images_from_directory(train_dir, label_value=0)  # Update label_value accordingly
test_images, test_labels = load_images_from_directory(test_dir, label_value=1)    # Update label_value accordingly

In [ ]:
from sklearn.model_selection import train_test_split

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(1000).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # Modify output layer based on your labels

base_model.trainable = False  # Freeze the base model

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

# Evaluate on the test set
test_loss, test_acc = model.evaluate(test_dataset)
print(f'Test accuracy: {test_acc}')

In [ ]:
import matplotlib.pyplot as plt

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()